# Import and Cleaning

In [17]:
# Required Libraries
import pandas as pd
import sqlite3


In [3]:
# Load CSVs
menu_df = pd.read_csv('Data/fastfood.csv')
menu_df.head(3)

,restaurant,item,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,Mcdonalds,Artisan Grilled Chicken Sandwich,380,60,7,2.0,0.0,95,1110,44,3.0,11,37.0,4.0,20.0,20.0,Other
1,Mcdonalds,Single Bacon Smokehouse Burger,840,410,45,17.0,1.5,130,1580,62,2.0,18,46.0,6.0,20.0,20.0,Other
2,Mcdonalds,Double Bacon Smokehouse Burger,1130,600,67,27.0,3.0,220,1920,63,3.0,18,70.0,10.0,20.0,50.0,Other


In [4]:
# Get information on df
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
 13  vit_a        301 non-null    float64
 14  vit_c        305 non-null    float64
 15  calcium      305 non-null    float64
 16  salad        515 non-null    object 
dtypes: float64(7), int64(7), object(3)
memory usage: 68.5+ KB


In [5]:
# Drop Columns missing several values
# Dropping Salad because it's not a useful field
menu_df = menu_df.drop(labels=['vit_a', 'vit_c', 'calcium', 'salad'], axis=1)

# Check to ensure it worked
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
dtypes: float64(4), int64(7), object(2)
memory usage: 52.4+ KB


In [6]:
# Clean Fiber and Protien missing values
# Calculate column averages
fiber_avg = menu_df['fiber'].mean()
protein_avg = menu_df['protein'].mean()

# Fill missing values with averages
menu_df['fiber'].fillna(fiber_avg, inplace=True)
menu_df['protein'].fillna(protein_avg, inplace=True)

# Verify the changes
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        515 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      515 non-null    float64
dtypes: float64(4), int64(7), object(2)
memory usage: 52.4+ KB


In [7]:
# Convert all integer columns to floats to make it easier to compare
int_columns = menu_df.select_dtypes(include='int64').columns
menu_df[int_columns] = menu_df[int_columns].astype(float)

# Create a new column to display the item with the chain name after, making it easier to read the drop down menus
menu_df.insert(2, 'item_with_chain', menu_df['item'] + ' (' + menu_df['restaurant'] + ')')

# Verify the changes
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant       515 non-null    object 
 1   item             515 non-null    object 
 2   item_with_chain  515 non-null    object 
 3   calories         515 non-null    float64
 4   cal_fat          515 non-null    float64
 5   total_fat        515 non-null    float64
 6   sat_fat          515 non-null    float64
 7   trans_fat        515 non-null    float64
 8   cholesterol      515 non-null    float64
 9   sodium           515 non-null    float64
 10  total_carb       515 non-null    float64
 11  fiber            515 non-null    float64
 12  sugar            515 non-null    float64
 13  protein          515 non-null    float64
dtypes: float64(11), object(3)
memory usage: 56.5+ KB


In [8]:
# Find duplicate items in 'item_with_chain'
print(menu_df['item_with_chain'].value_counts())

duplicate_items = menu_df[menu_df.duplicated(subset='item_with_chain', keep=False)]

# Display duplicate items
print(duplicate_items[['item_with_chain', 'calories', 'cal_fat', 'sat_fat', \
                       'trans_fat', 'cholesterol', 'sodium', 'total_carb', 'fiber', 'sugar', 'protein']])

# Remove one of the duplicates (you can choose 'first' or 'last')
menu_df = menu_df.drop_duplicates(subset='item_with_chain', keep='first')

item_with_chain
Express Taco Salad w/ Chips (Taco Bell)         2
Chili Cheese Burrito (Taco Bell)                2
Artisan Grilled Chicken Sandwich (Mcdonalds)    1
6" Sweet Onion Chicken Teriyaki (Subway)        1
Footlong Subway Seafood Sensation (Subway)      1
                                               ..
Roast Turkey & Swiss Sandwich (Arbys)           1
Roast Beef Gyro (Arbys)                         1
Reuben Sandwich (Arbys)                         1
5 piece Prime-Cut Chicken Tenders (Arbys)       1
Fiesta Taco Salad-Steak (Taco Bell)             1
Name: count, Length: 513, dtype: int64
                             item_with_chain  calories  cal_fat  sat_fat  \
414         Chili Cheese Burrito (Taco Bell)     380.0    150.0      8.0   
492         Chili Cheese Burrito (Taco Bell)     380.0    150.0      8.0   
497  Express Taco Salad w/ Chips (Taco Bell)     580.0    260.0      9.0   
511  Express Taco Salad w/ Chips (Taco Bell)     580.0    260.0      9.0   

     trans_fat

In [9]:
# Check again
duplicate_items2 = menu_df[menu_df.duplicated(subset='item_with_chain', keep=False)]
print(duplicate_items2)

Empty DataFrame
Columns: [restaurant, item, item_with_chain, calories, cal_fat, total_fat, sat_fat, trans_fat, cholesterol, sodium, total_carb, fiber, sugar, protein]
Index: []


In [11]:
# Look at shape
menu_df.shape

(513, 14)

In [12]:
# Look at columns
menu_df.columns

Index(['restaurant', 'item', 'item_with_chain', 'calories', 'cal_fat',
       'total_fat', 'sat_fat', 'trans_fat', 'cholesterol', 'sodium',
       'total_carb', 'fiber', 'sugar', 'protein'],
      dtype='object')

# Export to SQLite

In [19]:


# Connect to SQLite database (this will create a new database file if it doesn't exist)
conn = sqlite3.connect('Data/menu.db')

# Export the DataFrame to SQLite
menu_df.to_sql(name='menu', con=conn, index=False, if_exists='replace')

# Close the connection
conn.close()

## Testing importing the data again

In [22]:
# Connect to SQLite database
conn = sqlite3.connect('Data/menu.db')

# Query to retrieve data from the 'menu' table
query = 'SELECT * FROM menu'

# Use Pandas to read data from SQLite into a DataFrame
menu2_df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
menu2_df.head(2)

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
0,Mcdonalds,Artisan Grilled Chicken Sandwich,Artisan Grilled Chicken Sandwich (Mcdonalds),380.0,60.0,7.0,2.0,0.0,95.0,1110.0,44.0,3.0,11.0,37.0
1,Mcdonalds,Single Bacon Smokehouse Burger,Single Bacon Smokehouse Burger (Mcdonalds),840.0,410.0,45.0,17.0,1.5,130.0,1580.0,62.0,2.0,18.0,46.0


In [21]:
menu2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant       513 non-null    object 
 1   item             513 non-null    object 
 2   item_with_chain  513 non-null    object 
 3   calories         513 non-null    float64
 4   cal_fat          513 non-null    float64
 5   total_fat        513 non-null    float64
 6   sat_fat          513 non-null    float64
 7   trans_fat        513 non-null    float64
 8   cholesterol      513 non-null    float64
 9   sodium           513 non-null    float64
 10  total_carb       513 non-null    float64
 11  fiber            513 non-null    float64
 12  sugar            513 non-null    float64
 13  protein          513 non-null    float64
dtypes: float64(11), object(3)
memory usage: 56.2+ KB
